In [10]:
from math import floor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import solver
import Optimisation as op

ModuleNotFoundError: No module named 'solver'

In [11]:
earth = solver.Planet()
ans = op.plot_optimisation("../data/ChelyabinskEnergyAltitude_1.csv", earth)

NameError: name 'solver' is not defined

In [8]:
import scipy.interpolate as intpl
import pandas as pd
import matplotlib.pyplot as plt
'''
Optimization problem

'''

def meteor_data(file):
    """ The given results of the meteor event.
    """
    data_csv = pd.read_csv(file)
    df=pd.DataFrame(data=data_csv)
    altitude_ = df[df.columns[0]].to_numpy()
    energy = df[df.columns[1]].to_numpy() #* 4.184 * 1e12 # convert to joules
    return altitude_, energy

def RMSE(energy_guess, energy_target):
    """  Calculate the root mean square error of the optimized energy and target energy
    """
    return np.sqrt(np.mean((energy_guess-energy_target)**2))

# loop through each possible r and Y
def get_opt_radius_Y(rad, Y, height_ori, energy_ori, target_func):
    
    rmse_all = []
    tol = 5
 
    for r in rad:
        for s in Y:
            result = earth.solve_atmospheric_entry(radius = r, angle=18.3, strength = s, velocity=1.92e4, density=3300)
            outcome = earth.calculate_energy(result)

            energy_guess = outcome.dedz
            height_guess = outcome.altitude/1000
            
            # Slice optimized function to the same range as target one
            lower_bound = np.where(height_guess <= height_ori[0])[0][0]
            upper_bound = np.where(height_guess >= height_ori[-1])[0][-1]
            height_guess_s = height_guess[lower_bound:upper_bound]
            energy_guess_s = energy_guess[lower_bound:upper_bound]
            
            # Calculate optimal energy 
            energy_ori_s = target_func(height_guess_s)

            # Output energy rmse difference, put error into an array
            rmse = RMSE(energy_guess_s, energy_ori_s)
            rmse_all.append(rmse)
            print('smallest:')
            print(np.amin(rmse_all[:]))
            print('rmse:')
            print(rmse)

            if rmse < np.amin(rmse_all[:]) or np.allclose(rmse, np.amin(rmse_all[:])):
                radius_opt = r
                strength_opt = s
                print('Yay you are the chosen radius: %f m'%(radius_opt))
                print('Yay you are the smallest rmse: %f '%(rmse))

            elif rmse<tol:
                radius_opt = r
                strength_opt = s
                break
                
    return radius_opt, strength_opt,rmse, height_guess_s, energy_guess_s

def plot_optimisation(filepath_):
    height_ori, energy_ori = meteor_data(filepath_) # insert filename
    target_func = intpl.interp1d(height_ori, energy_ori)

    fig = plt.figure(figsize=(18, 6))
    ax = fig.add_subplot(121)
    # Interpolate function
    target_func = intpl.interp1d(height_ori, energy_ori)
    # Plot target function
    ax.plot(height_ori, target_func(height_ori),'r',label = 'Target func')

    #Guess energy and height
    result = earth.solve_atmospheric_entry(radius=8.21, angle=18.3, strength=5e6, velocity=1.92e4, density=3300)
    outcome = earth.calculate_energy(result)

    energy_guess = outcome.dedz
    height_guess = outcome.altitude/1000

    # Plot guess function
    ax.plot(height_guess, energy_guess,label = 'Guess func')
    ax.legend()
    ax.grid(True)
    ax.set_ylabel('Energy Loss per Unit Height (kt TNT)')
    ax.set_xlabel('Altitude (km)')


    # # Slice optimized function to the same range as target one
    # lower_bound = np.where(height_guess <= height_ori[0])[0][0]
    # upper_bound = np.where(height_guess >= height_ori[-1])[0][-1]
    # height_guess_s = height_guess[lower_bound:upper_bound]
    # energy_guess_s = energy_guess[lower_bound:upper_bound]

    # Change range ehere
    radius_ = np.linspace(8.0, 8.3, 3)
    strength_ = np.linspace(4.9e6,5.3e6, 3)

    radius_opt, strength_opt, rmse, height_guess_s, energy_guess_s = get_opt_radius_Y(radius_, strength_ ,
                                                                            height_ori, energy_ori, target_func)

    ax1 = plt.subplot(122)
    ax1.plot(height_guess_s, energy_guess_s, label = 'Guess func')
    ax1.plot(height_ori, target_func(height_ori),'r', label = 'Target func')
    ax1.grid(True)
    ax1.legend()
    ax1.set_ylabel('Energy Loss per Unit Height (kt TNT)')
    ax1.set_xlabel('Altitude (km)')
    
    return radius_opt, strength_opt, rmse
    # energy_ori_s = target_func(height_guess_s)

# rmse = RMSE(energy_guess_s, energy_ori_s)
plot_optimisation("../data/ChelyabinskEnergyAltitude_1.csv")
print('radius_opt:')
print(radius_opt)
print('strength_opt: ')
print(strength_opt)

FileNotFoundError: [Errno 2] File b'../data/ChelyabinskEnergyAltitude_1.csv' does not exist: b'../data/ChelyabinskEnergyAltitude_1.csv'

In [ ]:
radius_opt